In [1]:
import json
import numpy as np
import pandas as pd
import os

from math import cos, sin, exp, sqrt, pi, e

from src.OspitalettoDataset import OspitalettoDataset
from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

## Constants to be used across the notebook

In [2]:
simulation_values_path = os.path.join(".", "data", "simulation_values.json")    
with open(simulation_values_path, "r") as f:
    simulation_values = json.load(f)
    
simulation_values

{'DT_evap': 8.0,
 'Tdhw': 55.0,
 'Tmax_i': 60.0,
 'Tmin_i': 45.0,
 'Ts1': 23.0,
 'Ts2': 29.0,
 'depth_aquifer': 30.0,
 'n_buildings_MFH': 1600.0,
 'n_buildings_SFH': 0.0,
 's1_schedule': [[0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0]],
 's2_schedule': [[1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 1, 1, 1, 0]

In [3]:
load_or_write_new = int(input("Press 1 if you want to reload previous simulation values"))

if load_or_write_new == 1:    
    Tmin_i = simulation_values["Tmin_i"]
    Tmax_i = simulation_values["Tmax_i"]

    Tdhw = simulation_values["Tdhw"]

    Ts1 = simulation_values["Ts1"]
    Ts2 = simulation_values["Ts2"]

    DT_evap = simulation_values["DT_evap"]
    
    n_buildings_SFH = simulation_values["n_buildings_SFH"]
    n_buildings_MFH = simulation_values["n_buildings_MFH"]
    
else:
    # Min and Max temperatures to be received by the user.
    # Values [40,...,60]
    Tmin_i = float(input("Enter min setpoint temperature: "))
    Tmax_i = float(input("Enter max setpoint temperature: "))

    # Domestic hot water temperature.
    # Values [45,...,55]
    Tdhw = float(input("Enter domestic hot water temperature: "))

    # Our model supposes that the system has two different sources of heat.
    # Values [20,...,30]
    Ts1 = float(input("Temperature of source 1: "))
    Ts2 = float(input("Temperature of source 2: "))

    # difference in temperature between the supply (system->heat->house) and return (house -> heat -> system) pipes
    # Values [1,...,10]
    DT_evap = float(input("Temperature difference among the supply and return pipes: "))
    
    # For the data visualization course, we will select 0 SFH & 1600 MFH buildings, which will cover an area of 1 km2
    n_buildings_SFH =float(input("Enter the number of SFH buildings: #"))
    n_buildings_MFH=float(input("Enter the number of MFH buildings:#"))
    
print(f"* {Tmin_i=}\n"
      f"* {Tmax_i=}\n"
      f"* {Tdhw=}\n"
      f"* {Ts1=}\n"
      f"* {Ts2=}\n"
      f"* {DT_evap=}\n"
      f"* {n_buildings_SFH=}\n"
      f"* {n_buildings_MFH=}\n"
     )

Press 1 if you want to reload previous simulation values 1


* Tmin_i=45.0
* Tmax_i=60.0
* Tdhw=55.0
* Ts1=23.0
* Ts2=29.0
* DT_evap=8.0
* n_buildings_SFH=0.0
* n_buildings_MFH=1600.0



# Load Sources working hours

The file must be a boolean matrix $W$ of $24 x 7$. The rows represent the hours in a day, and the columns represent the days in the week. For all $i \in \{0,...,23\}$ and $j \in \{0,..., 6\}$ we have that $w_{i,j} \in \{0, 1\}$, where $w_{i,j} = 0$ means that the source doesn't produce energy at that hour $i$ on that day $j$. Similarly, $w_{i,j} = 1$ means that the source produces energy at that hour $i$ on that day $j$.

In [5]:
s1_schedule_path = os.path.join(".", "data", "private", "s1_source_fake_schedule.xlsx")
if load_or_write_new == 1:
    s1_schedule = np.array(simulation_values["s1_schedule"])
else:    
    s1_schedule = pd.read_excel(s1_schedule_path, index_col="Time").to_numpy()

print(f"* {s1_schedule_path=}\n"
      f"* {s1_schedule=}\n"
      f"* {s1_schedule[6,1]=}\n" # Tuesdays are closed at 6 (equal to 0)
      f"* {s1_schedule[7,1]=}\n") # but open are 7 (equal to 1)

* s1_schedule_path='./data/private/s1_source_fake_schedule.xlsx'
* s1_schedule=array([[0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 0]])
* s1_schedule[6,1]=0
* s1_schedule[7,1]=1



In [6]:
s2_schedule_path = os.path.join(".", "data", "private", "s2_source_fake_schedule.xlsx")
if load_or_write_new == 1:
    s2_schedule = np.array(simulation_values["s2_schedule"])
else: 
    s2_schedule = pd.read_excel(s2_schedule_path, index_col="Time").to_numpy()

print(f"* {s2_schedule_path=}\n"
      f"* {s2_schedule=}\n"
      f"* {s2_schedule[15,1]=}\n" # Tuesdays are closed at 15 (equal to 0)
      f"* {s2_schedule[16,1]=}\n") # but open are 16 (equal to 1)

* s2_schedule_path='./data/private/s2_source_fake_schedule.xlsx'
* s2_schedule=array([[1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 1, 0, 1, 1, 1, 0],
       [1, 0, 0, 1, 1, 1, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 0, 0]])
* s2_schedule[15,1]=1
* s2_schedule[16,1]=0



## Take note of the values for the current simulation

In [11]:
simulation_values["Tmin_i"] = Tmin_i
simulation_values["Tmax_i"] = Tmax_i
simulation_values["Tdhw"] = Tdhw
simulation_values["Ts1"] = Ts1
simulation_values["Ts2"] = Ts2
simulation_values["DT_evap"] = DT_evap
simulation_values["s1_schedule"] = s1_schedule.tolist()
simulation_values["s2_schedule"] = s2_schedule.tolist()
simulation_values["n_buildings_SFH"] = n_buildings_SFH
simulation_values["n_buildings_MFH"] = n_buildings_MFH
simulation_values

{'DT_evap': 8.0,
 'Tdhw': 55.0,
 'Tmax_i': 60.0,
 'Tmin_i': 45.0,
 'Ts1': 23.0,
 'Ts2': 29.0,
 'depth_aquifer': 30.0,
 'n_buildings_MFH': 1600.0,
 'n_buildings_SFH': 0.0,
 's1_schedule': [[0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 0, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 1, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0],
  [0, 1, 1, 1, 1, 0, 0]],
 's2_schedule': [[1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 0, 1, 1, 0],
  [1, 1, 0, 1, 1, 1, 0]

## Read the dataset

In [12]:
ospitaletto2019 = OspitalettoDataset()
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(ospitaletto2019.load_processed_data())
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.update(insPireDataset.load_processed_data())
AVAILABLE_DATASETS.keys()

dict_keys(['ospitaletto', 'miami_florida', 'fresno_california', 'olympia_washington', 'rochester_newyork', 'london_uk', 'madrid_spain', 'rome_italy', 'stuttgart_germany'])

In [13]:
dataset_to_work = "stuttgart_germany"
dataset_to_work

'stuttgart_germany'

In [10]:
Tamb_hourly = AVAILABLE_DATASETS[dataset_to_work]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.943899,0.0,0.0,9.952648,3.272608,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.070529,0.0,0.0,9.952648,3.421508,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.292571,0.0,0.0,9.952648,3.961412,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.279545,0.0,0.0,9.952648,3.962210,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.271487,0.0,0.0,9.952648,3.962704,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.358953,0.0,0.0,9.952648,3.293909,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.902323,0.0,0.0,9.952648,3.274109,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.814213,0.0,0.0,9.952648,3.277296,0.010184,-0.005816,0.004367


## Space Heating and Domestic Hot Water consumption

The user determines the number of buildings and building type (single family house or small-multi family house) supplied by a district heating network. Data from the Inspire project was used to estimate the annual energy consumption for each building typology in kWh/m2-y. SFHs and MFHs correspond to reference buildings in each climatic zone built between 1945-1970, non-refurbished. 

The SFH and MFH models have a fixed geometry for all the climates and periods of construction. It has been defined following the common characteristics for a European buildings. The SFH building model is composed of two storeys with a total of 100 m² of living area (Inspire project, D2.1c(2014)). The small-MFH correspond to a reference residential building that has two dwellings per floor, with a total floor area of 500 m2.

### Residential building typologies: SFH and MFH

#### Single-Family units

<img src="./images/SFH.png">        





#### Multi-Families units

<img src="./images/MFH.png">




### District heating system diagram

<img src="./images/DHC_system.png">

### Calculating SH, DWH and Heat Demand

This calculation is based on the number of SFH ($ |SFH| $) and MFH ($ |MFH| $) units. It also depends on the total heat needed by SFH ($ TOT_{SFH} $) and MFH ($ TOT_{MFH} $) units. The formula for the Heat Demand($ HD $) is the following  

\begin{align*}
HD = |SFH| * 100 * TOT_{SFH} + |MFH| * 500 * TOT_{MFH}
\end{align*}

It's important to stress that this calculation is done on a year basis.

In [14]:
Tamb_hourly["Heat_demand"] = (n_buildings_SFH * 100 * Tamb_hourly["SFH_bldg_tot"]) + (n_buildings_MFH * 500 * Tamb_hourly["MFH_bldg_tot"])
Tamb_hourly["SH_demand"] = Tamb_hourly["Heat_demand"] * Tamb_hourly["%SH_y"]
Tamb_hourly["DHW_demand"] = Tamb_hourly["Heat_demand"] * Tamb_hourly["%DHW_y"] # Equivalently, we can change Tamb_hourly["%DHW_y"] with (1 - Tamb_hourly["%SH_y"])
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.943899,0.0,0.0,9.952648,3.272608,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.070529,0.0,0.0,9.952648,3.421508,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.292571,0.0,0.0,9.952648,3.961412,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.279545,0.0,0.0,9.952648,3.962210,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.271487,0.0,0.0,9.952648,3.962704,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.358953,0.0,0.0,9.952648,3.293909,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.902323,0.0,0.0,9.952648,3.274109,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.814213,0.0,0.0,9.952648,3.277296,0.010184,-0.005816,0.004367


### Domestic hot water consumption

Domestic hot water consumption is not weather influenced, and its variation is almost constant over the year. The hourly demand for hot water is estimated by the percentage of the total heat demand ($ HD_y $) is hot water $ DHW_y $. This data is obtained from the INSPIRE database for different climatic zones. In this particular case, the climatic zones and reference cities considered are:

* Stuttgart, Germany for Continental climate
* London, United Kingdom for Oceanic climate
* Madrid, Spain for Southern-dry climate
* Rome, Italy for Mediterranean climate

The total domestic hot water demand is evenly distributed over each day of the year, and then its hourly distribution is obtained by multiplying the daily needs by an hourly random profile. 

\begin{align*}
DHW_{d} = \frac{HD_{y}*DHW_{\%y}}{d}
\end{align*}

The DHW consumption is calculated in an hourly fashion.

In [15]:
Tamb_hourly["DHW_consumption"] = Tamb_hourly["DHW_demand"] * Tamb_hourly["DHW_hourly_consumption_ratio"] / 365
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.943899,0.0,0.0,9.952648,3.272608,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.070529,0.0,0.0,9.952648,3.421508,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.292571,0.0,0.0,9.952648,3.961412,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.279545,0.0,0.0,9.952648,3.962210,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.271487,0.0,0.0,9.952648,3.962704,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.358953,0.0,0.0,9.952648,3.293909,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.902323,0.0,0.0,9.952648,3.274109,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.814213,0.0,0.0,9.952648,3.277296,0.010184,-0.005816,0.004367


### Space heating profile and heating degree hours

The amount (in degrees) and for how long (in hours) of thermal heat required to keep the indoor building temperature at a comfortable level will vary depending on different climates. The base temperature selected was set to 15 °C.

The space heating hourly profile for each location is then retrieved with a time dependency according to the heating degree method, known as the "Integration method".

More info on calculation method: https://www.degreedays.net/introduction

In [16]:
def heating_degree(T_amb):
    T_base = 15
    return max(0, T_base - T_amb)

In [17]:
Tamb_hourly["heating_degree_days"] = Tamb_hourly.air_temp.apply(heating_degree)
Tamb_hourly["space_heating_dist"] = Tamb_hourly.heating_degree_days / max(sum(Tamb_hourly.heating_degree_days), 1)
Tamb_hourly["SH_consumption"] = Tamb_hourly["SH_demand"] * Tamb_hourly["space_heating_dist"]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.943899,0.0,0.0,9.952648,3.272608,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.070529,0.0,0.0,9.952648,3.421508,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.292571,0.0,0.0,9.952648,3.961412,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.279545,0.0,0.0,9.952648,3.962210,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.271487,0.0,0.0,9.952648,3.962704,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.358953,0.0,0.0,9.952648,3.293909,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.902323,0.0,0.0,9.952648,3.274109,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.814213,0.0,0.0,9.952648,3.277296,0.010184,-0.005816,0.004367


### Total hourly consumption

This is as easy as 

\begin{align*}
CON_{TOT} = CON_{SH} + CON_{DHW}
\end{align*}



In [18]:
Tamb_hourly["Total_consumption"] = Tamb_hourly["DHW_consumption"] + Tamb_hourly["SH_consumption"]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.943899,0.0,0.0,9.952648,3.272608,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.070529,0.0,0.0,9.952648,3.421508,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.292571,0.0,0.0,9.952648,3.961412,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.279545,0.0,0.0,9.952648,3.962210,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.271487,0.0,0.0,9.952648,3.962704,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.358953,0.0,0.0,9.952648,3.293909,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.902323,0.0,0.0,9.952648,3.274109,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.814213,0.0,0.0,9.952648,3.277296,0.010184,-0.005816,0.004367


In [186]:
Tamb_hourly["SH_consumption"].shape

(8761,)



## Calculate Space Heating and Hot Water Temperature

The space Heating is calculated using a climatic curve. The Hot Water temperature is constant across the year.

In [21]:
def climatic_curve(Tamb_h):
    Tmin_o = 2.38 #minimum outdoor T threshold in which the space heating system turns on
    Tmax_o = 7.25 #maximum outdoor T threshold in which the space heating system turns off

    if Tamb_h <= Tmin_o:   
        Tsh = Tmax_i
    elif Tamb_h >= Tmax_o:
        Tsh = Tmin_i
    else:
        m = (Tmax_i-Tmin_i)/(Tmin_o-Tmax_o)
        b = -m*Tmin_o+Tmax_i
        Tsh = m*Tamb_h+b
    return Tsh
 

In [22]:
Tamb_hourly['space_heating_temp'] = Tamb_hourly.air_temp.apply(climatic_curve)
Tamb_hourly["hot_water_temp"] = Tdhw
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.943899,0.0,0.0,9.952648,3.272608,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.070529,0.0,0.0,9.952648,3.421508,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.292571,0.0,0.0,9.952648,3.961412,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.279545,0.0,0.0,9.952648,3.962210,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.271487,0.0,0.0,9.952648,3.962704,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.358953,0.0,0.0,9.952648,3.293909,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.902323,0.0,0.0,9.952648,3.274109,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.814213,0.0,0.0,9.952648,3.277296,0.010184,-0.005816,0.004367


## Calculate User Temperature

This is the temperature that the users receive by the system. The temperature is based on their personal preferences, and the current season of the year.

In [23]:
sh_consumption_ratio = Tamb_hourly["SH_consumption"] / Tamb_hourly["Total_consumption"]
dhw_consumption_ratio = Tamb_hourly["DHW_consumption"] / Tamb_hourly["Total_consumption"]

In [24]:
#Tamb_hourly["user_temp_2"] = Tamb_hourly.apply(Tuser, axis=1)
Tamb_hourly["user_temp"] = (sh_consumption_ratio * Tamb_hourly.space_heating_temp 
                            + dhw_consumption_ratio * Tamb_hourly.hot_water_temp)

Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.9439,0.0,0.0,9.952648,3.272608,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.0705,0.0,0.0,9.952648,3.421508,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2926,0.0,0.0,9.952648,3.961412,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2795,0.0,0.0,9.952648,3.962210,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2715,0.0,0.0,9.952648,3.962704,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.359,0.0,0.0,9.952648,3.293909,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.9023,0.0,0.0,9.952648,3.274109,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.8142,0.0,0.0,9.952648,3.277296,0.010184,-0.005816,0.004367


## Calculate Sources Temperature based on Working Hours

In [25]:
dayofweeks = Tamb_hourly.index.dayofweek
hours = Tamb_hourly.index.hour

In [26]:
Tamb_hourly["source1_temp"] = Ts1 * s1_schedule[hours, dayofweeks]
Tamb_hourly["source2_temp"] = Ts2 * s2_schedule[hours, dayofweeks]
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.9439,0.0,0.0,9.952648,3.272608,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.0705,0.0,0.0,9.952648,3.421508,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2926,0.0,0.0,9.952648,3.961412,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2795,0.0,0.0,9.952648,3.962210,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2715,0.0,0.0,9.952648,3.962704,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.359,0.0,0.0,9.952648,3.293909,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.9023,0.0,0.0,9.952648,3.274109,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.8142,0.0,0.0,9.952648,3.277296,0.010184,-0.005816,0.004367


## Calculate Network temperature

In [27]:
def calculate_tnet(temp_s1, temp_s2, temp_aq):
    if temp_s1 == 0.0 and temp_s2 == 0.0:
        return temp_aq
    elif temp_s1 == 0.0:
        return temp_s2
    elif temp_s2 == 0.0:
        return temp_s1
    else:
        return np.mean([temp_s1, temp_s2])

In [28]:
Tamb_hourly["net_temp"] = Tamb_hourly.apply(lambda fila: calculate_tnet(fila.source1_temp, fila.source2_temp, fila.aquifer_temp), axis=1)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.9439,0.0,0.0,9.952648,3.272608,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.0705,0.0,0.0,9.952648,3.421508,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2926,0.0,0.0,9.952648,3.961412,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2795,0.0,0.0,9.952648,3.962210,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2715,0.0,0.0,9.952648,3.962704,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.359,0.0,0.0,9.952648,3.293909,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.9023,0.0,0.0,9.952648,3.274109,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.8142,0.0,0.0,9.952648,3.277296,0.010184,-0.005816,0.004367


# Coefficient of performance (COP)

In [29]:
DT_hx = 2.5
n_HP = 0.53
Te_o = Tamb_hourly.net_temp - DT_evap - DT_hx
Tc_o = Tamb_hourly.user_temp

Tamb_hourly['COP']= 1-n_HP + n_HP * (Tc_o + 273.15) / (Tc_o + DT_hx - Te_o)
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.9439,0.0,0.0,9.952648,3.27261,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.0705,0.0,0.0,9.952648,3.42151,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2926,0.0,0.0,9.952648,3.96141,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2795,0.0,0.0,9.952648,3.96221,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2715,0.0,0.0,9.952648,3.9627,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.359,0.0,0.0,9.952648,3.29391,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.9023,0.0,0.0,9.952648,3.27411,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.8142,0.0,0.0,9.952648,3.2773,0.010184,-0.005816,0.004367


# Heat losses

In [31]:
def heat_losses(T_net,T_gr, DT_evap):
    U = 2
    T_ret = T_net - DT_evap
    HL_s = (T_net - T_gr) * U / 1000 # Heat losses supply pipe [MWh]
    HL_r = (T_ret - T_gr) * U / 1000 # Heat losses return pipe [MWh]
    
    return pd.Series([HL_s, HL_r, HL_s + HL_r] , index=['E_loss_s','E_loss_r','E_loss_tot'])


In [32]:
x = Tamb_hourly.apply(lambda fila: heat_losses(fila["net_temp"], fila["ground_temp"], DT_evap), axis=1, result_type='expand')

In [33]:
Tamb_hourly['E_loss_s'] = x['E_loss_s']
Tamb_hourly['E_loss_r'] = x['E_loss_r']
Tamb_hourly['E_loss_tot'] = x['E_loss_tot']
Tamb_hourly

,air_temp,dayofyear,month,hourofyear,air_temp_fit,ground_temp,aquifer_temp,city_key,climate,DHW_cons,...,space_heating_temp,hot_water_temp,user_temp,source1_temp,source2_temp,net_temp,COP,E_loss_s,E_loss_r,E_loss_tot
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-1.60,1,1,1,0.954843,4.855297,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.9439,0.0,0.0,9.952648,3.27261,0.010195,-0.005805,0.004389
2017-01-01 01:00:00,3.65,1,1,2,0.953185,4.852557,9.952648,stuttgart_germany,Continental,21.7,...,56.088295,55.0,56.0705,0.0,0.0,9.952648,3.42151,0.010200,-0.005800,0.004400
2017-01-01 02:00:00,8.75,1,1,3,0.951533,4.849819,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2926,0.0,0.0,9.952648,3.96141,0.010206,-0.005794,0.004411
2017-01-01 03:00:00,8.45,1,1,4,0.949885,4.847085,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2795,0.0,0.0,9.952648,3.96221,0.010211,-0.005789,0.004422
2017-01-01 04:00:00,8.25,1,1,5,0.948242,4.844353,9.952648,stuttgart_germany,Continental,21.7,...,45.000000,55.0,45.2715,0.0,0.0,9.952648,3.9627,0.010217,-0.005783,0.004433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 20:00:00,-1.45,365,12,8757,0.963198,4.866283,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.359,0.0,0.0,9.952648,3.29391,0.010173,-0.005827,0.004345
2017-12-31 21:00:00,-1.50,365,12,8758,0.961517,4.863532,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.9023,0.0,0.0,9.952648,3.27411,0.010178,-0.005822,0.004356
2017-12-31 22:00:00,-1.50,365,12,8759,0.959842,4.860784,9.952648,stuttgart_germany,Continental,21.7,...,60.000000,55.0,59.8142,0.0,0.0,9.952648,3.2773,0.010184,-0.005816,0.004367


# Electricity consumption

The hourly electricity consumption by climatic zone is estimated as a function of the total thermal demand and the coefficient of performance of the systems substations:

\begin{align*}
E_{el} = \frac{E_{th}}{COP}
\end{align*}

## Save CSV version of the dataset and simulation values

In [34]:
AVAILABLE_DATASET_PATHS = {OspitalettoDataset.OSPITALETTO: ospitaletto2019.processed_dataset_path,
                           NOAA2010Dataset.MIAMI_FL: noaa2010Dataset.processed_miami_fl_dataset_path, 
                           NOAA2010Dataset.FRESNO_CA: noaa2010Dataset.processed_fresno_ca_dataset_path, 
                           NOAA2010Dataset.OLYMPIA_WA: noaa2010Dataset.processed_olympia_wa_dataset_path,
                           NOAA2010Dataset.ROCHESTER_NY: noaa2010Dataset.processed_rochester_ny_dataset_path,
                           InsPireDataset.LONDON_UK: insPireDataset.processed_london_uk_dataset_path,
                           InsPireDataset.MADRID_SPA: insPireDataset.processed_madrid_spa_dataset_path,
                           InsPireDataset.ROME_IT: insPireDataset.processed_rome_it_dataset_path,
                           InsPireDataset.STUTTGART_GER: insPireDataset.processed_stuttgart_ger_dataset_path,}

Tamb_hourly.to_csv(path_or_buf=AVAILABLE_DATASET_PATHS[dataset_to_work])

In [35]:
simulation_values_path = os.path.join(".", "data", "simulation_values.json")

with open(simulation_values_path, "w") as f:
    json.dump(simulation_values, f, sort_keys=True, indent=2)